In [ ]:
### Investigating acceptance 

# imports
import numpy as np
import matplotlib.pyplot as plt
from functions import *

n = 1000
betas = [0.01, 0.05, 0.1, 0.5, 1]
Ds = [4, 16]
log_target = log_continuous_target
log_likelihood = log_continuous_likelihood

### Generate K, the covariance of the Gaussian process, and sample from N(0,K) using a stable Cholesky decomposition
def priorSamples(l):
	K = GaussianKernel(coords, l)
	z = np.random.randn(N, )
	Kc = np.linalg.cholesky(K + 1e-6 * np.eye(N))
	u = Kc @ z
	return u, K

grw_accept = []
pcn_accept = []

for d in Ds:

	Dx = d
	Dy = d
	N = Dx*Dy
	coords = [(x, y) for y in np.linspace(0,1,Dy) for x in np.linspace(0,1,Dx)]    # Coordinates for the inference grid
	subsample_factor = 4
	idx = subsample(N, subsample_factor)
	M = len(idx)  

	# generate samples with length scale 0.3
	u, K = priorSamples(0.3)
	Kc = np.linalg.cholesky(K + 1e-6 * np.eye(N))

	# Observation model: v = G(u) + e,   e~N(0,I)
	G = get_G(N, idx)
	v = G @ u + np.random.randn(M)
	u0 = Kc @  np.random.randn(N, )
	
	for b in betas:
		_, grw_acceptance = grw(log_target, u0, v, K, G, n, b)
		_, pcn_acceptance = pcn(log_likelihood, u0, v, K, G, n, b)
		grw_accept.append(grw_acceptance)
		pcn_accept.append(pcn_acceptance)

In [ ]:
print(grw_accept)
print(pcn_accept)